# 1.2 手続きとその生成するプロセス

・**プロセス** とは、一つの仕事を表す単位、または処理そのもののことである。（時田解釈）  
・プロセスの **局所的展開(local evolution)** は、手続きによって記述される。  
・手続きの全体的なふるまい、**グローバル(global)**なふるまいについて何か記述できるようにしたい。  

本節では、単純な手続きによって作られたプロセスの、いくつかの形を見ていく。  
具体的には、  
・線形再帰、反復、木の再帰・・・空間(メモリ)の消費  
・増加オーダー・・・時間の消費  
である。

## 1.2.1 線形再帰と反復

階乗関数について考える。定義は以下の通り。  
$ n!=n \cdot (n-1) \cdot (n-2) \cdots 3 \cdot 2 \cdot 1 $  

階乗の計算方法として、$ (n-1)! $を計算し、その計算結果に$ n $をかけることで$ n! $を計算できる。  
$ n!=n \cdot [(n-1) \cdot (n-2) \cdots 3 \cdot 2 \cdot 1]=n \cdot (n-1)! $  

これに$ 1!=1 $という規定を加えると、そのまま手続きとして実装できる。

In [1]:
(define (factorial n)
  (if (= n 1) 1
      (* n (factorial (- n 1))
         )
      )
  )

In [2]:
(factorial 6)

720

この手続きが$ 6! $の計算を実行する様子を見るには、1.1.5節の置換モデルを使うことができる。

### 図1.3 $ 6! $を求めるための線形再帰プロセス

    (factorial 6)
    (* 6 (factorial 5))
    (* 6 (* 5 (factorial 4)))
    (* 6 (* 5 (* 4 (factorial 3))))
    (* 6 (* 5 (* 4 (* 3 (factorial 2)))))
    (* 6 (* 5 (* 4 (* 3 (* 2 (factorial 1))))))
    (* 6 (* 5 (* 4 (* 3 (* 2 1)))))
    (* 6 (* 5 (* 4 (* 3 2))))
    (* 6 (* 5 (* 4 6)))
    (* 6 (* 5 24))
    (* 6 120)
    720


In [3]:
;階乗の計算の様子を出力させてみた。
(define (factorial2 n)
  (if (= n 1)
      (begin
       (display n)
       (display '!=)
       (display n)
       (newline)
       1)
       (begin
        (display n)
        (display '!=)
        (display n)
        (display '* )
        (display (- n 1))
        (display '!)
        (newline)
        (define ret (* n (factorial2 (- n 1))))
        (display n)
        (display '!=)
        (display ret)
        (newline)
        ret
        )
      )
  )

In [4]:
(factorial2 6)

6!=6*5!
5!=5*4!
4!=4*3!
3!=3*2!
2!=2*1!
1!=1
2!=2
3!=6
4!=24
5!=120
6!=720


720

In [5]:
;階乗の計算の様子を出力させてみた。
;上記だと(define ret ・・・)でエラーになる場合があった。
(define (factorial2 n)
  (if (= n 1)
      (begin
       (display n)
       (display '!=)
       (display n)
       (newline)
       1)
        (begin
          (display n)
          (display '!=)
          (display n)
          (display '* )
          (display (- n 1))
          (display '!)
          (newline)
          (let ((ret (* n (factorial2 (- n 1)))))
            (begin
              (display n)
              (display '!=)
              (display ret)
              (newline)
              ret
            )
          )
        )
    )
  )

In [6]:
(factorial2 6)

6!=6*5!
5!=5*4!
4!=4*3!
3!=3*2!
2!=2*1!
1!=1
2!=2
3!=6
4!=24
5!=120
6!=720


720

このプロセスの置換モデルは、図1.3にあるように展開・縮約という形をしている。  
展開は、プロセスが**遅延演算(deferred operation)**の連鎖(ここではかけ算の連鎖)を構築する際に起こる。  
縮約は、演算が実際に⾏われる際に起きる。  
遅延演算の連鎖によって特徴づけられるこのタイプのプロセスは、**再帰プロセス(recursive process)**と呼ばれる。  
このプロセスを実⾏するには、インタプリタは後で実⾏する演算について記録しておく必要がある。  
$n!$の計算では、遅延乗算の連鎖の⻑さ、またそれを記録するために必要な情報の量は、$n$に対して線形に ($n$に⽐例して)増加する。  
ステップ数も、同じように線形に増加する。  
このようなプロセスは、**線形再帰プロセス(linear recursive process)**と呼ばれる。  

階乗の別の計算方法を考えてみる。  
$ n! $の計算規則として、最初に$ 1 \times 2 $を行い、それからその答えに$ 3 $をかけ、それから$ 4 $をかけ、$ n $まで続けるというものである。  
形式的に書くと、計算途中の積と$ 1 $から$ n $まで数え上げていくカウンタを保持することで、次の規則に従ってカウンタと積がステップごとに同時に変化するように記述できる。  
$ product \leftarrow counter \times product $  
$ counter \leftarrow counter + 1 $  

$ n! $とはカウンタが$ n $を超えた時点での積の値であると規定すると、以下のように実装できる。

In [7]:
(define (factorial3 n)
  (fact-iter 1 1 n))
(define (fact-iter product counter max-count)
  (if (> counter max-count)
      product
      (fact-iter (* counter product) (+ counter 1) max-count)
    )
  )

In [8]:
(factorial3 6)

720

先ほどと同じく、置換モデルを使い、視覚化すると以下のようになる。

### 図1.4 $ 6! $を計算する線形反復プロセス

    (factorial 6)
    (fact-iter   1 1 6)
    (fact-iter   1 2 6)
    (fact-iter   2 3 6)
    (fact-iter   6 4 6)
    (fact-iter  24 5 6)
    (fact-iter 120 6 6)
    (fact-iter 720 7 6)


In [9]:
;階乗の計算の様子を出力させてみた。
(define (factorial4 n)
  (fact-iter2 1 1 n))
(define (fact-iter2 product counter max-count)
  (if (> counter max-count)
      (begin
       product
       )
      (begin
       (display "fact-iter2 ")
       (display product)
       (display " ")
       (display counter)
       (display " ")
       (display max-count)
       (newline)
       (fact-iter2 (* counter product) (+ counter 1) max-count)
       )
    )
  )

In [10]:
(factorial4 6)

fact-iter2 1 1 6
fact-iter2 1 2 6
fact-iter2 2 3 6
fact-iter2 6 4 6
fact-iter2 24 5 6
fact-iter2 120 6 6


720

1つ目のプロセスとは対照的に、⼆つ⽬のプロセスは伸びたり縮んだりしない。  
各ステップで記録しておく必要があるのは、どんな$n$に対しても、変数product,counter,max-countの現在の値だけである。  
これは**反復プロセス(iterative process)**と呼ぶ。  
⼀般的に、反復プロセスとは状態が限られた数の**状態変数(state variable)**に集約されるようなプロセスである。  
$n!$の計算では、必要なステップ数は$n$に対して線形に増加する。  
このようなプロセスは、**線形反復プロセス(linear iterative process)**と呼ばれる。

反復と再帰を⽐較する際に、**再帰プロセス(process)**という概念と**再帰⼿続き(procedure)**という概念を混同しないように気をつける必要がある。  
・⼿続きについて再帰と⾔うときは、その⼿続きの定義が(直接または間接的に)その⼿続き⾃⾝を参照しているという構⽂的事実を指している。  
・プロセスが例えば線形再帰パターンになっていると⾔う場合は、どのようにプロセスが展開するかということである。  

fact-iterのような再帰⼿続きが反復プロセスを⽣成すると⾔うのは、違和感があるかもしれない。  
だが、状態は三つの状態変数によって完全に表せるので反復プロセスである。  
このプロセスではインタプリタは三つの変数を記録しておくだけで実行することができる。   

⼀般的な⾔語(AdaやPascal、C⾔語を含む)のほとんどの実装では、   
再帰⼿続きの解釈には必ず⼿続き呼び出しの回数に⽐例してメモリを消費するように設計されているため、  
これらの言語ではプロセスと手続きの区別しにくい。  
(これらの言語での再帰手続きでは、スタックを使用して実装しているため、再帰プロセス・反復プロセスのいずれにしてもメモリを消費してしまう)  

そのため、このような⾔語では、反復⼿続きを記述する専⽤の  
**“ループ構造”** do, repeat, until, for, whileが用意されている。  
第5 章で考えるSchemeの実装には、この⽋点がない。
反復プロセスは、再帰⼿続きとして記述されていても、固定の空間(メモリ)で実⾏できる。
この性質を持った実装は**末尾再帰(tail-recursive)**と呼ばれる。


##### 練習問題
- [練習問題1.09 加算のプロセスの図示](../exercises/1.09.ipynb)
- [練習問題1.10 アッカーマン関数](../exercises/1.10.ipynb)